In [1]:
from tabulate import tabulate
import opendatasets as od
import pandas as pd
import os
import re

In [2]:
# Download the Kaggle data required
url = "https://www.kaggle.com/datasets/imdevskp/corona-virus-report/download?datasetVersionNumber=166"
od.download(url, force=True)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading corona-virus-report.zip to ./corona-virus-report


100%|██████████| 19.0M/19.0M [00:02<00:00, 9.20MB/s]


In [3]:
# Put in a list downloaded files
folder = "corona-virus-report"
covid_file_list = os.listdir(f"{folder}/")
print(covid_file_list)

['country_wise_latest.csv', 'covid_19_clean_complete.csv', 'day_wise.csv', 'full_grouped.csv', 'usa_county_wise.csv', 'worldometer_data.csv']


In [4]:
parquet_folder = "corona-virus-parquet-pandas"
# regex pattern for check dates
reg_date = re.compile("^\d{4}-\d{2}-\d{2}$")
reg_date2 = re.compile("^\d{1,2}\/\d{1,2}\/\d{2}$")
# Iterate files downloaded list
for file in covid_file_list:
    print(file)
    # Read file downloaded as DataFrame
    df = pd.read_csv(f"./{folder}/{file}", sep=',')
    # Check if Date column match with date regex, to format it
    if 'Date' in list(df.columns):
        print(df['Date'][0])
        if reg_date.match(df['Date'][0]):
            df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d")
        elif reg_date2.match(df['Date'][0]):
            df['Date'] = pd.to_datetime(df['Date'], format="%m/%d/%y")
    # Replace empty strings with None
    df = df.replace([''], [None])
    print(df.dtypes)
    print(tabulate(df.head(5), headers='keys', tablefmt='psql'))
    parquet_path = f"{parquet_folder}/{file.split('.')[0]}.parquet"
    # Saving as parquet
    df.to_parquet(parquet_path)

country_wise_latest.csv
Country/Region             object
Confirmed                   int64
Deaths                      int64
Recovered                   int64
Active                      int64
New cases                   int64
New deaths                  int64
New recovered               int64
Deaths / 100 Cases        float64
Recovered / 100 Cases     float64
Deaths / 100 Recovered    float64
Confirmed last week         int64
1 week change               int64
1 week % increase         float64
WHO Region                 object
dtype: object
+----+------------------+-------------+----------+-------------+----------+-------------+--------------+-----------------+----------------------+-------------------------+--------------------------+-----------------------+-----------------+---------------------+-----------------------+
|    | Country/Region   |   Confirmed |   Deaths |   Recovered |   Active |   New cases |   New deaths |   New recovered |   Deaths / 100 Cases |   Recovered / 100 C